# INIT

In [ ]:
import numpy as np
import pandas as pd
import math

from scipy import stats as s
from statistics import mode

from sklearn.metrics import confusion_matrix

In [ ]:
!gdown --id '1reRYfySXuRytE24CCgsINpGdC4X4NnPl' -O data_daun_a.csv
!gdown --id '1uYgW4LLLrS3NmMmfBlUFa6Nifk_CPD17' -O data_daun_b.csv
!gdown --id '1klhM0w98dpdIO-vUjn0S8WsLznmLU5FD' -O data_daun_c.csv
!gdown --id '1FnKNeqJ7sYLEM608NxiyRTilG_eL2N4i' -O data_daun_d.csv

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1reRYfySXuRytE24CCgsINpGdC4X4NnPl
To: /content/data_daun_a.csv
100% 409/409 [00:00<00:00, 670kB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1uYgW4LLLrS3NmMmfBlUFa6Nifk_CPD17
To: /content/data_daun_b.csv
100% 401/401 [00:00<00:00, 663kB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https

In [ ]:
DAUN_A = pd.read_csv('data_daun_a.csv').to_numpy() # Konversi DataFrame ke Numpy Array
DAUN_B = pd.read_csv('data_daun_b.csv').to_numpy()
DAUN_C = pd.read_csv('data_daun_c.csv').to_numpy()
DAUN_D = pd.read_csv('data_daun_d.csv').to_numpy()

data_daun = np.array([DAUN_A, DAUN_B, DAUN_C, DAUN_D]) # menyatukan semua daun ke dalam suatu array

DAUN = ["A", "B", "C", "D"]

# FUNCTIONS

In [ ]:
def euclidean(x1,x2):
  return np.sqrt(np.sum((x1-x2)**2))

def cosinus(x1,x2):
  total_atas = np.sum(x1*x2) 
  total_bawah = math.sqrt(np.sum(np.square(x1))) * math.sqrt(np.sum(np.square(x2))) 
  return total_atas / total_bawah

def knn(X,K = 3,D = 5,reverse=False):
  # X: Data Training, K: N pada KNN, D: Rentang Database
  if reverse :
    KX = np.floor(np.argsort(X)[::-1]/D)
  else:
    KX = np.floor(np.argsort(X)/D)
  NKX = KX[:K]
  return s.mode(NKX)[0]
  # return mode(NKX)

def knn_euclidean(X,Y,N = 3,D = 5):
    JX = []
    # out = ['A','B','C','D']
    for i in range(len(Y)):
      JX.append(euclidean(X,Y[i]))
    JX  = np.array(JX)
    output = int(knn(JX,N,D))
    if output == 0:
      return 'A'
    elif output == 1:
      return 'B'
    elif output == 2:
      return 'C'
    elif output >= 3:
      return 'D'

def knn_cosinus(X,Y,N = 3,D = 5):
    JX = []
    out = ['A','B','C','D']
    for i in range(len(Y)):
      JX.append(cosinus(X,Y[i]))
    JX  = np.array(JX)
    return out[int(knn(JX,N,D,reverse=True))]

def accuracy(val,expect):
  error = 0
  for i in range(len(val)):
    if val[i] != expect[i]:
      error += 1

  return ((len(val)-error)/len(val))*100

In [ ]:
def folding(data, num_of_fold, average=False):
  sections = [] # tempat untuk menyimpan setiap input dan database pada suatu folding
  """
  sections = [
    #1 [input, db],
    #2 [input, db],
    ...
  ]
  """
  temp_daun = [] # tempat sementara untuk menyimpan daun2 yang sudah di-split berdasar besaran folding
  """
  temp_daun = [
    daunA [daunA[0:5], daunA[5:10, ...]],
    daunB [daunB[0:5], daunB[5:10, ...]],
    ...
  ]
  """
  for i in range(len(data)):
    temp_daun.append([])

  for index, daun in enumerate(data):
    base = int(data.shape[1]/num_of_fold)
    for i in range(0, len(daun), base):
      temp_daun[index].append(daun[i:i+base])

  # for loop diatas digunakan untuk menjadikan array temp_daun seperti yang tertera diatas.

  for i in range(num_of_fold):
    sections.append([])

  for index in range(len(sections)):
    temp_input = []
    temp_db = []
    for jenis, daun in enumerate(temp_daun):
      temp_input.append(daun[index])
      if average == True:
        temp_db.append(np.average(np.concatenate(([ daun[i] for i in range(num_of_fold) if i != index] ), axis=0), axis=0))
      else:
        temp_db.append(np.concatenate(([ daun[i] for i in range(num_of_fold) if i != index] ), axis=0))
    sections[index].append(temp_input)  # input
    sections[index].append(temp_db) # database

  # for loop diatas digunakan untuk menjadikan array sections seperti yang tertera diatas.

  return sections

In [ ]:
def confusion_m(actual, predicted, label=["A", "B", "C", "D"]):
  c = confusion_matrix(actual,predicted)
  df = pd.DataFrame(data=c, columns=label, index=label)
  return df

In [ ]:
from functools import reduce

def summary_confusion_m(matrixes):
  return reduce(lambda a, b: a.add(b, fill_value=0), matrixes)

In [ ]:
def sum_conf_m(matrixes):
  temp = df.copy()
  result = temp.pop(0)
  for df in temp:
    result = result.add(df, fill_value=0)
  return result

# Implement

## 2 FOLD

In [ ]:
num_of_folds = 2
num_of_k = 20

folds = folding(data_daun, num_of_folds)
rentang_data = 10

# 2 fold, rentang data = 10
expected_10 = "A"*10 + "B"*10 + "C"*10 + "D"*10
expected_10 = [*expected_10]

In [ ]:
k = [num for num in range(num_of_k) if num % 2 == 1]
outx = []

"""
struktur outx = [[[]]], [i][j][k]
i = k ke berapa?
j = fold ke berapa?
k = data output
"""

# looping k
for i in range(len(k)):
  outx.append([])
  print('\n\n++++++++++++++++')
  print("\nK : ",k[i])
  df = [] # untuk summary confusion matrix

  # init data input dan db sesuai dengan fold
  for j, fold in enumerate(folds):
    outx[i].append([])
    input = np.vstack(fold[0])
    db = np.vstack(fold[1])

    # perhitungan knn-euclidean
    for x in input:
      outx[i][j].append(knn_euclidean(x,db,k[i],rentang_data))
  
    # confusion matrix 
    y_true = expected_10
    y_pred = outx[i][j]
    print("\nfold ke: ", j+1)
    df.append(confusion_m(y_true, y_pred))
    print(df[j])

  print('\nSummary conf_m')
  print(summary_confusion_m(df))
  # print(sum_conf_m(df))
  all_val = np.hstack(np.array(outx[i]))
  print('\nAccuracy :', accuracy(all_val, expected_10*num_of_folds))
  print('++++++++++++++++')




++++++++++++++++

K :  1

fold ke:  1
   A   B  C  D
A  9   0  0  1
B  0  10  0  0
C  0   0  8  2
D  4   0  4  2

fold ke:  2
   A   B  C  D
A  5   0  0  5
B  0  10  0  0
C  0   0  5  5
D  0   0  4  6

Summary conf_m
    A   B   C  D
A  14   0   0  6
B   0  20   0  0
C   0   0  13  7
D   4   0   8  8

Accuracy : 68.75
++++++++++++++++


++++++++++++++++

K :  3

fold ke:  1
    A   B  C  D
A  10   0  0  0
B   0  10  0  0
C   0   0  5  5
D   5   0  2  3

fold ke:  2
   A   B  C  D
A  9   0  0  1
B  0  10  0  0
C  0   0  3  7
D  0   0  4  6

Summary conf_m
    A   B  C   D
A  19   0  0   1
B   0  20  0   0
C   0   0  8  12
D   5   0  6   9

Accuracy : 70.0
++++++++++++++++


++++++++++++++++

K :  5

fold ke:  1
    A   B  C  D
A  10   0  0  0
B   0  10  0  0
C   0   0  6  4
D   4   0  3  3

fold ke:  2
   A   B  C  D
A  9   0  0  1
B  0  10  0  0
C  0   0  3  7
D  0   0  4  6

Summary conf_m
    A   B  C   D
A  19   0  0   1
B   0  20  0   0
C   0   0  9  11
D   4   0  7   9

Accuracy

## 4 FOLD

In [ ]:
num_of_folds = 4

folds = folding(data_daun, num_of_folds)
rentang_data = 5

expected_5 = "A"*5 + "B"*5 + "C"*5 + "D"*5
expected_5 = [*expected_5]

In [ ]:
k = [num for num in range(num_of_k) if num % 2 == 1]
outx = []

# looping k
for i in range(len(k)):
  outx.append([])
  print('\n\n++++++++++++++++')
  print("\nK : ",k[i])
  df = [] # untuk summary confusion matrix

  # init data input dan db sesuai dengan fold
  for j, fold in enumerate(folds):
    outx[i].append([])
    input = np.vstack(fold[0])
    db = np.vstack(fold[1])

    # perhitungan knn-euclidean
    for x in input:
      outx[i][j].append(knn_euclidean(x,db,k[i],rentang_data))
  
    # confusion matrix 
    y_true = expected_5
    y_pred = outx[i][j]
    print("\nfold ke: ", j+1)
    df.append(confusion_m(y_true, y_pred))
    print(df[j])

  print('\nSummary conf_m')
  print(summary_confusion_m(df))
  # print(sum_conf_m(df))
  all_val = np.hstack(np.array(outx[i]))
  print('\nAccuracy :', accuracy(all_val, expected_5*num_of_folds))
  print('++++++++++++++++')




++++++++++++++++

K :  1

fold ke:  1
   A  B  C  D
A  2  0  3  0
B  0  0  0  5
C  0  0  0  5
D  0  1  1  3

fold ke:  2
   A  B  C  D
A  2  0  2  1
B  0  0  0  5
C  0  0  0  5
D  2  0  0  3

fold ke:  3
   A  B  C  D
A  0  2  1  2
B  0  0  0  5
C  0  0  0  5
D  0  0  0  5

fold ke:  4
   A  B  C  D
A  1  1  2  1
B  0  0  0  5
C  0  0  0  5
D  0  0  0  5

Summary conf_m
   A  B  C   D
A  5  3  8   4
B  0  0  0  20
C  0  0  0  20
D  2  1  1  16

Accuracy : 26.25
++++++++++++++++


++++++++++++++++

K :  3

fold ke:  1
   A  B  C  D
A  2  0  3  0
B  0  0  0  5
C  0  0  0  5
D  1  0  2  2

fold ke:  2
   A  B  C  D
A  5  0  0  0
B  0  0  0  5
C  0  0  0  5
D  2  0  0  3

fold ke:  3
   A  B  C  D
A  3  1  1  0
B  0  0  0  5
C  0  0  0  5
D  0  0  0  5

fold ke:  4
   A  B  C  D
A  4  0  1  0
B  0  0  0  5
C  0  0  0  5
D  0  0  0  5

Summary conf_m
    A  B  C   D
A  14  1  5   0
B   0  0  0  20
C   0  0  0  20
D   3  0  2  15

Accuracy : 36.25
++++++++++++++++


++++++++++++++++

K :  

## 5 FOLD

In [ ]:
num_of_folds = 5

folds = folding(data_daun, num_of_folds)
rentang_data = 4

expected_4 = "A"*rentang_data + "B"*rentang_data + "C"*rentang_data + "D"*rentang_data
expected_4 = [*expected_4]

In [ ]:
k = [num for num in range(num_of_k) if num % 2 == 1]
outx = []

# looping k
for i in range(len(k)):
  outx.append([])
  print('\n\n++++++++++++++++')
  print("\nK : ",k[i])
  df = [] # untuk summary confusion matrix

  # init data input dan db sesuai dengan fold
  for j, fold in enumerate(folds):
    outx[i].append([])
    input = np.vstack(fold[0])
    db = np.vstack(fold[1])

    # perhitungan knn-euclidean
    for x in input:
      outx[i][j].append(knn_euclidean(x,db,k[i],rentang_data))
  
    # confusion matrix 
    y_true = expected_4
    y_pred = outx[i][j]
    print("\nfold ke: ", j+1)
    df.append(confusion_m(y_true, y_pred))
    print(df[j])

  print('\nSummary conf_m')
  print(summary_confusion_m(df))
  # print(sum_conf_m(df))
  all_val = np.hstack(np.array(outx[i]))
  print('\nAccuracy :', accuracy(all_val, expected_4*num_of_folds))
  print('++++++++++++++++')




++++++++++++++++

K :  1

fold ke:  1
   A  B  C  D
A  3  0  1  0
B  0  0  0  4
C  0  0  0  4
D  0  0  0  4

fold ke:  2
   A  B  C  D
A  1  1  1  1
B  0  0  0  4
C  0  0  0  4
D  0  0  0  4

fold ke:  3
   A  B  C  D
A  0  2  0  2
B  0  0  0  4
C  0  0  0  4
D  1  1  0  2

fold ke:  4
   A  B  C  D
A  0  3  0  1
B  0  0  0  4
C  0  0  0  4
D  0  0  0  4

fold ke:  5
   A  B  C  D
A  0  0  0  4
B  0  0  0  4
C  0  0  0  4
D  0  0  0  4

Summary conf_m
   A  B  C   D
A  4  6  2   8
B  0  0  0  20
C  0  0  0  20
D  1  1  0  18

Accuracy : 27.500000000000004
++++++++++++++++


++++++++++++++++

K :  3

fold ke:  1
   A  B  C  D
A  3  0  1  0
B  0  0  0  4
C  0  0  0  4
D  2  0  0  2

fold ke:  2
   A  B  C  D
A  2  1  1  0
B  0  0  0  4
C  0  0  0  4
D  0  0  0  4

fold ke:  3
   A  B  C  D
A  4  0  0  0
B  0  0  0  4
C  0  0  0  4
D  2  0  0  2

fold ke:  4
   A  B  C  D
A  1  2  0  1
B  0  0  0  4
C  0  0  0  4
D  0  0  0  4

fold ke:  5
   A  B  C  D
A  1  1  0  2
B  0  0  0  4
C  0 